In [2]:
import UnityPy
from PIL import Image
from pathlib import Path
import os
import shutil

In [3]:
# Load
env = UnityPy.load(f'../data/ui_jacket_001686.ab')

In [4]:
env

In [5]:
def print_structure(obj, path="root"):
    if isinstance(obj, dict):
        for key, value in obj.items():
            current_path = f"{path}.{key}"
            print_structure(value, current_path)
    elif isinstance(obj, list):
        for index, value in enumerate(obj):
            current_path = f"{path}[{index}]"
            print_structure(value, current_path)
    else:
        print(f"{path}: {obj}")

def recursive_string_replace(obj, old, new):
    if isinstance(obj, dict):
        for key, value in obj.items():
            obj[key] = recursive_string_replace(value, old, new)
    elif isinstance(obj, list):
        for index, value in enumerate(obj):
            obj[index] = recursive_string_replace(value, old, new)
    elif isinstance(obj, tuple):
        new_tuple = []
        for index, value in enumerate(obj):
            new_tuple.append(recursive_string_replace(value, old, new))
        obj = tuple(new_tuple)
    elif isinstance(obj, str):
        if old in obj:
            obj = obj.replace(old, new)
    return obj


In [110]:

music_id = '002000'
env = UnityPy.load(f'../data/ui_jacket_001686_s'
                   f'.ab')
for obj in env.objects:
    print(obj.type.name)
    
    if obj.type.name == 'Sprite':
        data = obj.read()
        data.type_tree.m_Name = data.type_tree.m_Name.replace('_001686', f'_{music_id}')
        
        data.m_Name = data.m_Name.replace('_001686', f'_{music_id}')
        data.container = data.container.replace('_001686', f'_{music_id}')
        data.save()
        
    if obj.type.name == 'AssetBundle':
        data = obj.read()
        data.type_tree.m_AssetBundleName = data.type_tree.m_AssetBundleName.replace('_001686', f'_{music_id}')
        data.type_tree.m_Name = data.type_tree.m_Name.replace('_001686', f'_{music_id}')
        
        data.m_Name = data.m_Name.replace('_001686', f'_{music_id}')
        
        key = next(iter(data.m_Container))
        data.m_Container[key.replace('_001686', f'_{music_id}')] = data.m_Container[key]
        del data.m_Container[key]
        
        
        data.save()
        
    if obj.type.name == 'Texture2D':
        
        data = obj.read()
        data.save()
        
with open(f'./ui_jacket_{music_id}.ab', 'wb') as f:
    f.write(env.file.save())

test_env = UnityPy.load(f'./ui_jacket_{music_id}.ab')
for obj in test_env.objects:
    print(f'{obj.type.name} ==============================================')
    # if obj.type.name == 'AssetBundle':
    tree = obj.read_typetree()
    print_structure(tree)

Texture2D
Sprite
AssetBundle


KeyboardInterrupt: 

In [ ]:
def replace_value(obj, target_value, new_value, path="", visited=None):
    if visited is None:
        visited = set()

    # Prevent infinite recursion on circular references
    if id(obj) in visited:
        return obj
    visited.add(id(obj))

    if isinstance(obj, dict):
        new_dict = {}
        for key, value in obj.items():
            new_key = new_value if key == target_value else key
            current_path = f"{path}['{new_key}']" if path else str(new_key)
            if value == target_value:
                new_dict[new_key] = new_value
                print(f'{current_path} = {new_value} # (replaced)')
            else:
                new_dict[new_key] = replace_value(value, target_value, new_value, current_path, visited)
        return new_dict
    elif isinstance(obj, list):
        for index, item in enumerate(obj):
            current_path = f"{path}[{index}]"
            if item == target_value:
                obj[index] = new_value
                print(f'{current_path} = {new_value} # (replaced)')
            else:
                obj[index] = replace_value(item, target_value, new_value, current_path, visited)
        return obj
    elif isinstance(obj, tuple):
        new_tuple = []
        for index, item in enumerate(obj):
            current_path = f"{path}[{index}]"
            if item == target_value:
                new_tuple.append(new_value)
                print(f'{current_path} = {new_value} # (replaced)')
            else:
                new_tuple.append(replace_value(item, target_value, new_value, current_path, visited))
        return tuple(new_tuple)
    elif isinstance(obj, set):
        new_set = set()
        for item in obj:
            current_path = f"{path}"
            if item == target_value:
                new_set.add(new_value)
                print(f'{current_path}{item} = {new_value} # (replaced)')
            else:
                new_set.add(replace_value(item, target_value, new_value, current_path, visited))
        return new_set
    else:
        for attribute_name in dir(obj):
            if attribute_name.startswith('__') and attribute_name.endswith('__'):
                continue

            attribute_value = getattr(obj, attribute_name)
            current_path = f"{path}.{attribute_name}" if path else attribute_name
            if attribute_value == target_value:
                setattr(obj, attribute_name, new_value)
                print(f'{current_path} = {new_value} # (replaced)')
            else:
                replace_value(attribute_value, target_value, new_value, current_path, visited)
        return obj

In [17]:
music_id = '002000'

for sub_name in ['', '_s']:
    if sub_name == '_s':
        new_m_path_id = int(music_id) + 10000000
        m_path_id_1 = -630615454984241609
        m_path_id_2 = -9068764124424944558
        cab_hash = '72f38b5f3114e28901b11d48eb6d85e6'
        new_cab_hash = f'{new_m_path_id}'
    else:
        new_m_path_id = int(music_id)
        m_path_id_1 = 739899962197238906
        m_path_id_2 = -1066836184272249663
        cab_hash = '86ca849a3b53af73406c67412dda69d5'
        new_cab_hash = f'{new_m_path_id}'
        
        
    env = UnityPy.load(f'../data/ui_jacket_001686{sub_name}.ab')
    for obj in env.objects:
        print(obj.type.name)
        
        tree = obj.read_typetree()
        recursive_string_replace(tree, '_001686', f'_{music_id}')
        
        # if obj.type.name == 'Sprite':
        #     tree['m_RenderDataKey']['first']['data[0]'] = new_m_path_id
        #     tree['m_RenderDataKey']['first']['data[1]'] = new_m_path_id
        #     tree['m_RenderDataKey']['first']['data[2]'] = new_m_path_id
        #     tree['m_RenderDataKey']['first']['data[3]'] = new_m_path_id
        #     tree['m_RD']['texture']['m_PathID'] = new_m_path_id
    
        # if obj.type.name == 'AssetBundle':
        #     tree['m_PreloadTable'][0]['m_PathID'] = -new_m_path_id
        #     tree['m_Container'][1][1]['asset']['m_PathID'] = -new_m_path_id
        #     tree['m_PreloadTable'][1]['m_PathID'] = new_m_path_id
        #     tree['m_Container'][0][1]['asset']['m_PathID'] = new_m_path_id
        
        if obj.type.name == 'Texture2D':
            tree['m_StreamData']['path'] = f'archive:/CAB-{new_cab_hash}/CAB-{new_cab_hash}.resS'
            
        obj.save_typetree(tree)
    
    # env.file.container[f'assets/assetbundle/jacket{sub_name}/ui_jacket_001686{sub_name}.png'].assets_file.assetbundle.m_PreloadTable[1].path_id = new_m_path_id # (replaced)
    # env.file.container[f'assets/assetbundle/jacket{sub_name}/ui_jacket_001686{sub_name}.png'].assets_file.environment.objects[1].path_id = new_m_path_id # (replaced)
    # env.file.container[f'assets/assetbundle/jacket{sub_name}/ui_jacket_001686{sub_name}.png'].path_id = new_m_path_id # (replaced)
    # env.file.container[f'assets/assetbundle/jacket{sub_name}/ui_jacket_001686{sub_name}.png'].assets_file.assetbundle.m_Container[0][1].asset._obj.path_id = -new_m_path_id # (replaced)
    # env.file.container[f'assets/assetbundle/jacket{sub_name}/ui_jacket_001686{sub_name}.png'].assets_file.assetbundle.m_Container[0][1].asset.path_id = -new_m_path_id # (replaced)
    # env.file.container[f'assets/assetbundle/jacket{sub_name}/ui_jacket_001686{sub_name}.png'].assets_file.assetbundle.m_PreloadTable[0].path_id = -new_m_path_id # (replaced)
    # 
    # env.file = replace_value(env.file, m_path_id_1, new_m_path_id)
    # env.file = replace_value(env.file, m_path_id_2, -new_m_path_id)
    
    env.file.name = env.file.name.replace('_001686', f'_{music_id}')
    
    key = next(iter(env.file.container))
    env.file.container[key.replace('_001686', f'_{music_id}')] = env.file.container[key]
    del env.file.container[key]
    
    # CAB-##
    env.file.files[f'CAB-{new_cab_hash}'] = env.file.files[f'CAB-{cab_hash}']
    env.file.files[f'CAB-{new_cab_hash}'].assetbundle.m_Name = env.file.files[f'CAB-{new_cab_hash}'].assetbundle.m_Name.replace('_001686', f'_{music_id}')
    del env.file.files[f'CAB-{cab_hash}']
    
    env.file.files[f'CAB-{new_cab_hash}.resS'] = env.file.files[f'CAB-{cab_hash}.resS']
    del env.file.files[f'CAB-{cab_hash}.resS']
            
    with open(f'./ui_jacket_{music_id}{sub_name}.ab', 'wb') as f:
        f.write(env.file.save())
    
    test_env = UnityPy.load(f'./ui_jacket_{music_id}{sub_name}.ab')
    print(test_env)
    
    
        
for obj in test_env.objects:
    pass
#   print(f'{obj.type.name} ==============================================')
#   # if obj.type.name == 'AssetBundle':
#    tree = obj.read_typetree()
#    print_structure(tree)

Sprite
AssetBundle
Texture2D
Texture2D
Sprite
AssetBundle


In [ ]:
#music_id = '002000'
env = UnityPy.load(f'../data/ui_jacket_001686_s.ab')

In [89]:
music_id_s = int(music_id) + 10000000

for obj in env.objects:
    print(obj.type.name)
    
    tree = obj.read_typetree()
    recursive_string_replace(tree, '_001686', f'_{music_id}')
    
    if obj.type.name == 'Sprite':
        tree['m_RenderDataKey']['first']['data[0]'] = int(music_id_s)
        tree['m_RenderDataKey']['first']['data[1]'] = int(music_id_s)
        tree['m_RenderDataKey']['first']['data[2]'] = int(music_id_s)
        tree['m_RenderDataKey']['first']['data[3]'] = int(music_id_s)
        tree['m_RD']['texture']['m_PathID'] = int(music_id_s)

    if obj.type.name == 'AssetBundle':
        tree['m_PreloadTable'][0]['m_PathID'] = -int(music_id_s)
        tree['m_Container'][1][1]['asset']['m_PathID'] = -int(music_id_s)
        tree['m_PreloadTable'][1]['m_PathID'] = int(music_id_s)
        tree['m_Container'][0][1]['asset']['m_PathID'] = int(music_id_s)
    
    if obj.type.name == 'Texture2D':
        tree['m_StreamData']['path'] = f'archive:/CAB-{music_id_s}/CAB-{music_id_s}.resS'
        
    obj.save_typetree(tree)
inspect_object(env, -1066836184272249663, )

# env.file.files[f'CAB-86ca849a3b53af73406c67412dda69d5'].assetbundle.m_Name = env.file.files[f'CAB-86ca849a3b53af73406c67412dda69d5'].assetbundle.m_Name.replace('_001686', f'_{music_id}')

key = next(iter(env.file.container))
env.file.container[key.replace('_001686', f'_{music_id}')] = env.file.container[key]
del env.file.container[key]
# 
env.file.name = env.file.name.replace('_001686', f'_{music_id}')
env.file.files[f'CAB-{music_id_s}'] = env.file.files['CAB-72f38b5f3114e28901b11d48eb6d85e6']
del env.file.files['CAB-72f38b5f3114e28901b11d48eb6d85e6']
env.file.files[f'CAB-{music_id_s}'].assetbundle.m_Name = env.file.files[f'CAB-{music_id_s}'].assetbundle.m_Name.replace('_001686', f'_{music_id}')


env.file.files[f'CAB-{music_id_s}.resS'] = env.file.files['CAB-72f38b5f3114e28901b11d48eb6d85e6.resS']
del env.file.files['CAB-72f38b5f3114e28901b11d48eb6d85e6.resS']

with open(f'./ui_jacket_{music_id}.ab', 'wb') as f:
    f.write(env.file.save())


test_env = UnityPy.load(f'./ui_jacket_{music_id}.ab')
##for obj in test_env.objects:
#    print(f'{obj.type.name} ==============================================')
 #   # if obj.type.name == 'AssetBundle':
#    tree = obj.read_typetree()
#    print_structure(tree)

Texture2D
Sprite
AssetBundle


In [82]:
env.file.files

{'CAB-72f38b5f3114e28901b11d48eb6d85e6': <SerializedFile>,
 'CAB-72f38b5f3114e28901b11d48eb6d85e6.resS': <UnityPy.streams.EndianBinaryReader.EndianBinaryReader_Memoryview_BigEndian at 0x2a46e9cd070>}

In [ ]:
def isiterable(p_object):
    try:
        it = iter(p_object)
    except TypeError: 
        return False
    return True

def inspect_object(obj, target, replacement, visited=None):
    if visited is None:
        visited = set()

    # Prevent infinite recursion on circular references
    if id(obj) in visited:
        return
    visited.add(id(obj))

    if isiterable(obj):
        for key, value in obj.items():
            inspect_object(value, target, replacement, visited)
    else:
        for attribute_name in dir(obj):
            if attribute_name.startswith('__') and attribute_name.endswith('__'):
                continue

            attribute_value = getattr(obj, attribute_name)
            if isinstance(attribute_value, type(target)) and target == attribute_value:
                try:
                    setattr(obj, attribute_name, replacement)     
                    print(1)
                except:
                    pass

            if isinstance(attribute_value, (dict, list, tuple, set, object)):
                inspect_object(attribute_value, target, replacement, visited)

In [78]:
inspect_object(test_env, -1066836184272249663, )

In [169]:
def find_path(directory: str, extensions: list):
    path = Path(directory)
    
    for file in path.rglob('*'):
        if file.suffix.lower() in extensions:
            return file
    
    return None

In [123]:
# iterate over internal objects
for obj in env.objects:
    # process specific object types
    if obj.type.name == 'Texture2D':
        # parse the object data
        data = obj.read()
        image_path = find_path('../example',  ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.svg'])
        data.image = Image.open(image_path).resize(data.image.size).convert('RGB').convert('RGBA')
        data.save()

with open('./test.ab', 'wb') as f:
    f.write(env.file.save())
        

# 정리 및 테스트

In [164]:
def create_jacket(in_path: str, out_path: str, music_id: str):
    
    for jacket_sub_name in ['', '_s']:
        env = UnityPy.load(f'../data/ui_jacket_001686{jacket_sub_name}.ab')
        for obj in env.objects:
            if obj.type.name == 'Texture2D':
                data = obj.read()
                image_path = find_path(in_path,  ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.svg'])
                data.image = Image.open(image_path).resize(data.image.size).convert('RGB').convert('RGBA')
                data.save()
            
            tree = obj.read_typetree()
            recursive_string_replace(tree, f'_001686', f'_{music_id}')
            obj.save_typetree(tree)
            
        os.makedirs(f'{out_path}/AssetBundleImages/jacket{jacket_sub_name}', exist_ok=True)
        
        with open(f'{out_path}/AssetBundleImages/jacket{jacket_sub_name}/ui_jacket_{music_id}{jacket_sub_name}.ab', 'wb') as f:
            f.write(env.file.save())


In [165]:
shutil.rmtree('./output', ignore_errors=True)

In [166]:
os.makedirs('./output' , exist_ok=True)

create_jacket('../example', './output', '002000')

In [167]:
env = UnityPy.load(f'../data/ui_jacket_001686.ab')
for obj in env.objects:
    print('-----------------------')
    tree = obj.read_typetree()
    print_structure(tree)

-----------------------
root.m_Name: UI_Jacket_001686
root.m_Rect.x: 0.0
root.m_Rect.y: 0.0
root.m_Rect.width: 400.0
root.m_Rect.height: 400.0
root.m_Offset.x: 0.0
root.m_Offset.y: 0.0
root.m_Border.x: 0.0
root.m_Border.y: 0.0
root.m_Border.z: 0.0
root.m_Border.w: 0.0
root.m_PixelsToUnits: 100.0
root.m_Pivot.x: 0.5
root.m_Pivot.y: 0.5
root.m_Extrude: 1
root.m_IsPolygon: False
root.m_RenderDataKey.first.data[0]: 1649677243
root.m_RenderDataKey.first.data[1]: 1261876480
root.m_RenderDataKey.first.data[2]: 3169460618
root.m_RenderDataKey.first.data[3]: 30573560
root.m_RenderDataKey.second: 21300000
root.m_SpriteAtlas.m_FileID: 0
root.m_SpriteAtlas.m_PathID: 0
root.m_RD.texture.m_FileID: 0
root.m_RD.texture.m_PathID: 739899962197238906
root.m_RD.alphaTexture.m_FileID: 0
root.m_RD.alphaTexture.m_PathID: 0
root.m_RD.m_SubMeshes[0].firstByte: 0
root.m_RD.m_SubMeshes[0].indexCount: 6
root.m_RD.m_SubMeshes[0].topology: 0
root.m_RD.m_SubMeshes[0].baseVertex: 0
root.m_RD.m_SubMeshes[0].firstVerte

In [168]:
r_env = UnityPy.load(f'./output/ui_jacket_002000.ab')
# r_env = UnityPy.load(f'./output/ui_jacket_002000_s.ab')

for obj in r_env.objects:
    print('-----------------------' + obj.type.name)
    tree = obj.read_typetree()
    print_structure(tree)


-----------------------Sprite
root.m_Name: UI_Jacket_002000
root.m_Rect.x: 0.0
root.m_Rect.y: 0.0
root.m_Rect.width: 400.0
root.m_Rect.height: 400.0
root.m_Offset.x: 0.0
root.m_Offset.y: 0.0
root.m_Border.x: 0.0
root.m_Border.y: 0.0
root.m_Border.z: 0.0
root.m_Border.w: 0.0
root.m_PixelsToUnits: 100.0
root.m_Pivot.x: 0.5
root.m_Pivot.y: 0.5
root.m_Extrude: 1
root.m_IsPolygon: False
root.m_RenderDataKey.first.data[0]: 1649677243
root.m_RenderDataKey.first.data[1]: 1261876480
root.m_RenderDataKey.first.data[2]: 3169460618
root.m_RenderDataKey.first.data[3]: 30573560
root.m_RenderDataKey.second: 21300000
root.m_SpriteAtlas.m_FileID: 0
root.m_SpriteAtlas.m_PathID: 0
root.m_RD.texture.m_FileID: 0
root.m_RD.texture.m_PathID: 739899962197238906
root.m_RD.alphaTexture.m_FileID: 0
root.m_RD.alphaTexture.m_PathID: 0
root.m_RD.m_SubMeshes[0].firstByte: 0
root.m_RD.m_SubMeshes[0].indexCount: 6
root.m_RD.m_SubMeshes[0].topology: 0
root.m_RD.m_SubMeshes[0].baseVertex: 0
root.m_RD.m_SubMeshes[0].firs